In [1]:
import pandas as pd
import numpy as np

companies = pd.read_csv('./merged/new_merged_companies.csv')  # Replace 'your_companies.csv' with the path to your companies file

In [4]:
missing_founded_at_count = companies['founded_at'].isnull().sum()
missing_closed_at_count = companies['closed_at'].isnull().sum()

print(missing_founded_at_count, missing_closed_at_count)

companies.info()

105326 184604
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196554 entries, 0 to 196553
Data columns (total 64 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   id                         196554 non-null  object 
 1   entity_type                196554 non-null  object 
 2   name                       196553 non-null  object 
 3   category_code              123187 non-null  object 
 4   status                     196554 non-null  object 
 5   founded_at                 91228 non-null   object 
 6   closed_at                  11950 non-null   object 
 7   domain                     196554 non-null  int64  
 8   homepage_url               196554 non-null  int64  
 9   twitter_username           196554 non-null  int64  
 10  short_description          196554 non-null  int64  
 11  description                196554 non-null  int64  
 12  overview                   196554 non-null  int64  
 13  tag_list       

In [3]:
new_aq = pd.read_csv('./merged/new_acquisitions.csv')

new_aq['acquired_at'] = pd.to_datetime(new_aq['acquired_at'], errors='coerce')

acquisition_dates = pd.Series(new_aq.acquired_at.values, index=new_aq.acquired_object_id).to_dict()

# Iterate through the main companiesset and update 'closed_at' where necessary
for index, row in companies.iterrows():
    # Check if the company is marked as 'acquired' and 'closed_at' is missing
    if row['status'] == 'acquired' and pd.isnull(row['closed_at']):
        # Look up the 'acquired_at' date using 'id'
        acquired_at = acquisition_dates.get(row['id'])
        # If an 'acquired_at' date is found, update 'closed_at'
        if acquired_at:
            companies.at[index, 'closed_at'] = acquired_at

# Verify the updates by displaying the first few rows of the companiesframe
companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196554 entries, 0 to 196553
Data columns (total 64 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   id                         196554 non-null  object 
 1   entity_type                196554 non-null  object 
 2   name                       196553 non-null  object 
 3   category_code              123187 non-null  object 
 4   status                     196554 non-null  object 
 5   founded_at                 91228 non-null   object 
 6   closed_at                  11950 non-null   object 
 7   domain                     196554 non-null  int64  
 8   homepage_url               196554 non-null  int64  
 9   twitter_username           196554 non-null  int64  
 10  short_description          196554 non-null  int64  
 11  description                196554 non-null  int64  
 12  overview                   196554 non-null  int64  
 13  tag_list                   81

In [5]:
condition = companies['closed_at'].isna() & (companies['status'] == 'closed')

# Compute the maximum date among 'last_investment_at', 'last_funding_at', and 'updated_at' for these rows
max_dates = companies.loc[condition, ['last_investment_at', 'last_funding_at']].max(axis=1)

# Update 'closed_at' for these rows with the computed maximum date
companies.loc[condition, 'closed_at'] = max_dates

# Verify if the changes have been applied by displaying rows where 'closed_at' was updated
# companies[condition].head()

missing_founded_at_count = companies['closed_at'].isnull().sum()
missing_founded_at_count


184604

In [28]:
condition = companies['closed_at'].isna()
companies.loc[condition, 'closed_at'] = companies['updated_at'].max()

con = companies['founded_at'].isna()
companies.loc[con, 'founded_at'] = companies['created_at'].min()

companies.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196554 entries, 0 to 196553
Data columns (total 64 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   id                         196554 non-null  object        
 1   entity_type                196554 non-null  object        
 2   name                       196553 non-null  object        
 3   category_code              123187 non-null  object        
 4   status                     196554 non-null  object        
 5   founded_at                 196554 non-null  datetime64[ns]
 6   closed_at                  196554 non-null  datetime64[ns]
 7   domain                     196554 non-null  int64         
 8   homepage_url               196554 non-null  int64         
 9   twitter_username           196554 non-null  int64         
 10  short_description          196554 non-null  int64         
 11  description                196554 non-null  int64   

In [23]:
condition = companies['founded_at'].isna()

min_dates = companies.loc[condition, ['first_investment_at', 'first_funding_at', 'created_at']].min(axis=1)

# Update 'founded_at' for rows where 'closed_at' is greater than the computed minimum date using the correct iteration method
for index, min_date in min_dates.items():  # Using .items() for Series
    if pd.notna(companies.at[index, 'closed_at']) and companies.at[index, 'closed_at'] >= min_date:
        companies.at[index, 'founded_at'] = min_date

# Verify the changes by displaying rows where 'founded_at' was supposed to be updated
companies[condition].head()

,id,entity_type,name,category_code,status,founded_at,closed_at,domain,homepage_url,twitter_username,...,num_of_fin_org_financed,fin_org_financed,funded_object_id_y,num_of_person_financed,person_financed,funded_object_id,num_of_companies_financed,startup_financed,log_fund_tot,tot_fin
1,c:10,Company,Flektor,games_video,acquired,NaT,2007-05-30,1,1,1,...,NaN,0,NaN,NaN,0,NaN,NaN,0,0.0,0
2,c:100,Company,There,games_video,acquired,NaT,2005-05-29,1,1,1,...,NaN,0,NaN,NaN,0,NaN,NaN,0,0.0,0
7,c:100042,Company,HostNine,NaN,operating,NaT,NaT,1,1,1,...,NaN,0,NaN,NaN,0,NaN,NaN,0,0.0,0
8,c:10005,Company,Thomas Publishing,advertising,operating,NaT,NaT,1,1,1,...,NaN,0,NaN,NaN,0,NaN,NaN,0,0.0,0
21,c:10016,Company,Camcorder-batteries.us,other,operating,NaT,NaT,1,1,1,...,NaN,0,NaN,NaN,0,NaN,NaN,0,0.0,0


In [24]:
companies.info()
founded_at_empty = companies[companies['founded_at'].isna()]
founded_at_empty.to_csv('./merged/founded_at_empty.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196554 entries, 0 to 196553
Data columns (total 64 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   id                         196554 non-null  object        
 1   entity_type                196554 non-null  object        
 2   name                       196553 non-null  object        
 3   category_code              123187 non-null  object        
 4   status                     196554 non-null  object        
 5   founded_at                 152485 non-null  datetime64[ns]
 6   closed_at                  71711 non-null   datetime64[ns]
 7   domain                     196554 non-null  int64         
 8   homepage_url               196554 non-null  int64         
 9   twitter_username           196554 non-null  int64         
 10  short_description          196554 non-null  int64         
 11  description                196554 non-null  int64   

In [31]:
# Ensure 'public_at' and 'founded_at' are in datetime format
companies['public_at'] = pd.to_datetime(companies['public_at'], errors='coerce')

# Size before dropping IPOs before 2008
size_before_ipo = companies.shape[0]

# Drop rows for status = 'ipo' and 'public_at' before 2008
ipo_condition = (companies['status'] == 'ipo') & (companies['public_at'] < '2008-01-01')
companies = companies.drop(companies[ipo_condition].index)

# Size after dropping IPOs before 2008
size_after_ipo = companies.shape[0]

# Now, drop rows for every other status where 'founded_at' is before 2008
non_ipo_condition = (companies['status'] != 'ipo') & (companies['founded_at'] < '2008-01-01')
companies = companies.drop(companies[non_ipo_condition].index)

# Size after dropping non-IPOs founded before 2008
size_after_non_ipo = companies.shape[0]

size_before_ipo, size_after_ipo, size_after_non_ipo


(196554, 196267, 115706)

In [6]:
# Extract the year from 'closed_at' and 'founded_at'
companies['closed_at_year'] = companies['closed_at'].dt.year
companies['founded_at_year'] = companies['founded_at'].dt.year

# Calculate the age of the companies
companies['company_age'] = companies['closed_at_year'] - companies['founded_at_year']

# Display a few rows to verify the 'company_age' calculation
companies[['closed_at_year', 'founded_at_year', 'company_age']].head()


AttributeError: Can only use .dt accessor with datetimelike values

In [ ]:
companies.to_csv('./merged/aged_comp2008.csv', index=False)